In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
import pickle
import pandas as pd

In [2]:

##Load the trained model, scaler pickle, onehot
model=load_model('model.h5')

##Load the encoder and scaler
with open('one_hot_encoder_geo.pkl', 'rb') as file:
    Onehot_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [ ]:
# Example input data
input_data= {'CreditScore':600,
             'Geography': 'France',
             'Gender': 'Male',
             'Age': 40,
             'Tenure': 3,
             'Balance': 60000,
             'NumOfProducts': 2,
             'HasCrCard': 1,
             'IsActiveMember': 1,
             'EstimatedSalary': 50000
             }

In [13]:
geo_encoded = Onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded, columns=Onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/Users/iti/Upskilling/ML_Bootcamp_Krish_Naik/ANN_Project/my_practice/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [14]:
input_df=pd.DataFrame([input_data])

In [15]:
input_df=pd.concat([input_df.drop('Geography', axis=1),geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,800,Male,40,3,600000,2,1,1,500000,1.0,0.0,0.0


In [16]:
## Encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,800,1,40,3,600000,2,1,1,500000,1.0,0.0,0.0


In [17]:
##Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[ 1.54816798,  0.91025899,  0.09692036, -0.69444952,  8.37780421,
         0.79839749,  0.64900815,  0.97554884,  6.94941317,  0.99128795,
        -0.57869742, -0.56926697]])

In [18]:
## Predict Churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


array([[0.23640345]], dtype=float32)

In [19]:
#prediction probability
prediction_proba=prediction[0][0]
prediction_proba

np.float32(0.23640345)

In [20]:
if prediction_proba>0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
